In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamWordCount")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/24 22:05:52 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/24 22:05:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/24 22:05:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# pre-requistes

# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connect ot a server running on port 9999

# terminal, 

# nc -lk 9999

In [4]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

22/05/24 22:06:02 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

root
 |-- value: string (nullable = true)



In [6]:
from pyspark.sql.functions import split, explode, col
# split , split the string using space, convert line into word list
# explode project every element in list as element in stream [similar to flatMap]
# output is a stream of word
# Input "welcome to Kafka"
# split => ["welcome", "to", "Kafka"]
# explode makes every element as stream element
# welcome
# to
# Kafka
wordsDf = lineStreamDf.select(explode(split(lineStreamDf.value, " ")).alias("word"))
wordsDf.printSchema()

root
 |-- word: string (nullable = true)



In [7]:
# word count by group
# word  count
# Kafka  2..
wordCountDf = wordsDf.groupBy("word").count()
wordCountDf.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [8]:
# print out on console 
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
# check this cell or next cell
# Complete mode print all results for every change, even new word addd or existing word count updated 
# echoOnConsole = wordCountDf.writeStream.outputMode("complete").format("console").start()

In [9]:
# print out on console 
# Output: update
# only print if there is difference in output, not printing all results
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
echoOnConsole = wordCountDf.writeStream.outputMode("update").format("console").start()

22/05/24 22:06:07 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8aac6ca8-fef4-45ab-afb9-00984f69cd47. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    2|
|spark|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|spark|    2|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|java|    1|
+----+-----+

